In this notebook the code to calculate the surface energy of the bulk structures is ran.

This notebook is meant to run the HPC optimized version 8


Add your keys as environment variables

In [ ]:
import os
from getpass import getpass
os.environ['MPRESTER_API_KEY'] = getpass('MP REST API Key: ')
os.environ['HF_TOKEN'] = getpass('Hugging Face Token: ')


Lets delete the previously saved database:

In [3]:
import os
import shutil

db_file = 'surface_formation_energies_uma.db'
if os.path.exists(db_file):
    try:
        os.remove(db_file)
        print(f"Deleted {db_file}")
    except PermissionError:
        print(f"Permission denied, trying alternative...")
        shutil.rmtree(db_file, ignore_errors=True)
        print(f"Force deleted {db_file}")
else:
    print(f"{db_file} doesn't exist (clean state)")

# STEP 2: Verify it's gone
import glob
remaining = glob.glob('surface_formation_energies_uma*')
if remaining:
    print(f"Found related files: {remaining}")
else:
    print(f"Clean state confirmed")

surface_formation_energies_uma.db doesn't exist (clean state)
Clean state confirmed


Run the script

In [ ]:
from combined_pipeline_v8 import CombinedSurfaceEnergyCalculator

calc = CombinedSurfaceEnergyCalculator(
    input_db_file='S_filtered_structures.db',
    bulk_csv_file='bulk_energies_summary.csv',
    output_db='surface_formation_energies_uma.db',
    output_traj_dir='grouped_slab_trajectories_relaxed_structures_fixed',
    save_trajectories=True,
    save_slab_geometries=True,
    min_slab_size=10.0,
    min_vacuum=20.0,
    max_structures=None #Modify this to limit the number of processed structures - for testing purposes
)

results = calc.compute_all()

c:\Users\lucas\miniconda3\envs\uma_py310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
W1203 16:11:24.020000 26076 site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
2025-12-03 16:11:24,173 - INFO - ================================================================================
2025-12-03 16:11:24,173 - INFO - SURFACE FORMATION ENERGY CALCULATION
2025-12-03 16:11:24,173 - INFO - Model: UMA
2025-12-03 16:11:24,173 - INFO - ================================================================================
2025-12-03 16:11:24,173 - INFO - Loaded 31 bulk energies
2025-12-03 16:11:24,173 - INFO - Initializing UMA calculator...
2025-12-03 16:11:26,262 - INFO - HTTP Request: GET https://huggingface.co/api/whoami-v2 "HTTP/1.1 200 OK"
N

If the above cell does not work, run the diagnostic:


In [6]:
import sqlite3
import json

conn = sqlite3.connect('surface_formation_energies_uma.db')
cursor = conn.cursor()

# Get first entry
cursor.execute("SELECT id, key_value_pairs FROM systems LIMIT 1")
row = cursor.fetchone()

if row:
    row_id, kvp_text = row
    print(f"Row {row_id} key_value_pairs:")
    print(f"  Type: {type(kvp_text)}")
    print(f"  Length: {len(kvp_text)}")
    print(f"  Content: {kvp_text}")
    
    # Parse as JSON
    try:
        data = json.loads(kvp_text)
        print(f"\n✓ Parsed as JSON")
        print(f"  Keys: {list(data.keys())}")
        print(f"  Full content: {data}")
    except Exception as e:
        print(f"\n✗ Parse error: {e}")

conn.close()


Row 1 key_value_pairs:
  Type: <class 'str'>
  Length: 401
  Content: {"structure_idx": 0, "miller_index": "(1, 0, 0)", "termination_index": 0, "composition": "{\"Zr\": 20, \"Os\": 12}", "n_atoms": 32, "n_layers": 8, "n_fixed_atoms": 8, "slab_area_ang2": 46.28384989871256, "bulk_reference_energy_ev": -288.514652, "total_slab_energy_ev": -286.81938243714274, "surface_formation_energy_ev_per_ang2": 0.018313834810276153, "calculation_date": "2025-12-03T16:11:40.918219"}

✓ Parsed as JSON
  Keys: ['structure_idx', 'miller_index', 'termination_index', 'composition', 'n_atoms', 'n_layers', 'n_fixed_atoms', 'slab_area_ang2', 'bulk_reference_energy_ev', 'total_slab_energy_ev', 'surface_formation_energy_ev_per_ang2', 'calculation_date']
  Full content: {'structure_idx': 0, 'miller_index': '(1, 0, 0)', 'termination_index': 0, 'composition': '{"Zr": 20, "Os": 12}', 'n_atoms': 32, 'n_layers': 8, 'n_fixed_atoms': 8, 'slab_area_ang2': 46.28384989871256, 'bulk_reference_energy_ev': -288.514652, 'tota